# Building NLP Applications with Hugging Face

Welcome! In this project, you will be learning how to perform common Natural Language Processing (NLP) tasks using Hugging Face. Some of these tasks include:
- **sentiment analysis** (i.e. categorizing text as negative or positive);
- **text embedding** (i.e. transforming a piece of text into a numerical, n-dimensional vector, representation);
- **semantic search** (i.e. matching a query with the most appropriate result based on embeddings);
- and more!

The dataset comes from "Rent the Runway" [link](https://cseweb.ucsd.edu//~jmcauley/datasets.html#clothing_fit) and is comprised of user reviews on clothing items, their ratings on fit, and other metadata about the user (i.e. gender, height, size, age, reason for renting) and the item (i.e. category). It is a nice mixture of data types, but most importantly, lots of text! 

In order to be successful, you should have:

**Intermediate knowledge of Python**
- list comprehension
- for loops and while loops
- installing packages
- creating and using functions
- using NumPy and Pandas

**Basic understanding of NLP**
- What it is
- Data preparation steps and why they're important
- Familiarity, though not necessarily expert proficiency, in some NLP tasks

**Brief usage of Hugging Face**


Most of all, you should have a curiosity about NLP workflows, specifically those in Hugging Face using transformers!

## Task 0: Setup

For this project, we will need several Python packages:
- `pandas`
- `numpy`
- `datetime`
- `re`
- `string`
- `matplotlib.pyplot`
- `seaborn`
- `transformers`
- `sentence_transformers`

These packages will help us with the data preprocessing steps, visualization, and, of course, NLP tasks using Hugging Face (i.e. `transformers` and `sentence_transformers`).

### Instructions

Import the following packages.

- Import `re`, `datetime`, and `string`.
- Import `pandas` using the alias `pd`.
- Import `numpy` using the alias `np`.
- Import `matplotlib.pyplot` using the alias `plt`.
- From the `transformers` package, import `pipeline`.
- From the `sentence_transformers` package, import `SentenceTransformer`.
- From the `sentence_transformers.util` package, import `semantic_search`. 
- From the `IPython.display` package, import `display` and `Markdown`.

In [5]:
!pip install sentence-transformers

     ---------------------------------------- 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 977.6/977.6 kB 2.1 MB/s eta 0:00:00

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.0.2 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.2.5 which is incompatible.
spacy-transformers 1.0.2 requires transformers<4.6.0,>=3.4.0, but you have transformers 4.36.2 which is incompatible.



   ---------------------------------------- 1.1/1.1 MB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 192.3/192.3 MB 1.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=54ad707c34dfa489190f90f01f61ea2d522b0a0c19d3aba949ca516189dc0533
  Stored in directory: c:\users\josem\appdata\local\pip\cache\wheels\5e\6f\8c\d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0


In [6]:
# Import the other required packages and modules.
import re
import datetime
import string
import pandas as pd
import numpy as np
import matplotlib as plt
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import semantic_search

# From the IPython.display package, import display and Markdown
from IPython.display import display, Markdown

## Task 1: Import the Runway Data

The runway data is contained in a CSV file named `runway.csv`.

The dataset contains the following columns.

- `user_id`: the unique identifier for the user.
- `item_id`: the unique identifier for the item/product rented.
- `rating`: the rating by the user.
- `rented_for`: the reason the item was rented.
- `review_text`: the actual text for the submitted user review.
- `category`: the category of the item rented.
- `height`: the height of the user in the format {feet}'{inches}".
- `size`: the size of the item rented by the user.
- `age`: the age of the user.
- `review_date`: the date the review was made by the user.

### Instructions

Import the runway data to a pandas dataframe.

- Read the data from `runway.csv`, making sure to parse the date column. |Assign to `runway`.
- Print the column info

In [8]:
# Read the data from runway.csv
runway = pd.read_csv("runway.csv")

# Print the column info
print(runway.columns)

Index(['user_id', 'item_id', 'rating', 'rented for', 'review_text', 'category',
       'height', 'size', 'age', 'review_date'],
      dtype='object')


In [9]:
runway.describe()

,user_id,item_id,rating,size,age
count,1506.000000,1.506000e+03,1506.000000,1506.000000,1500.000000
mean,498540.554449,8.423255e+05,9.102258,12.492696,33.663333
std,291071.125644,6.057790e+05,1.395354,8.749235,7.950354
min,772.000000,1.233730e+05,2.000000,0.000000,15.000000
25%,249571.000000,1.815240e+05,8.000000,8.000000,29.000000
50%,495970.500000,8.100440e+05,10.000000,12.000000,32.000000
75%,754884.250000,1.373701e+06,10.000000,16.000000,37.000000
max,998134.000000,1.991314e+06,10.000000,58.000000,116.000000


In [10]:
runway.head()

,user_id,item_id,rating,rented for,review_text,category,height,size,age,review_date
0,476109,139086,8,formal affair,it hit the floor perfectly with a pair of heel...,gown,"5' 3""",15,27.0,Dec 19 2017
1,203660,1126889,6,party,the dress is absolutely gorgeous unfortunately...,dress,"5' 4""",12,28.0,Jan 03 2022
2,868581,652189,8,wedding,even though it was lined with satin this was a...,dress,"5' 5""",24,30.0,Aug 05 2021
3,935076,1879504,8,wedding,this dress was greatit fit really well and was...,sheath,"5' 3""",14,37.0,Oct 02 2021
4,995023,1179146,10,party,super flattering i am usually a sizemi have a ...,dress,"5' 2""",14,37.0,Nov 20 2022


## Task 2: Preprocessing the `review_text`

Most unstructured text, such as reviews for products, are messy. They contain special characters which may not be necessary, extra spaces, irrelevant digits, and more. Therefore, it is common practice to process, or clean, the text before performing NLP tasks on it.

You will create several processing steps for the `review_text` strings.

Note: there may be some instances where special characters, digits, and the like are important to the meaning, or context, of the sentence. It's best to think through the implications of such preprocessing steps before blindly doing so.

Also note: some Pythonistas will say preprocessing text before using transformers, such as those from Hugging Face, is unnecessary. We will explore this in the following tasks. 

### Instructions

Create several steps that will preprocess the text for the following areas:
- replace the special character "/" with a space;
- remove punctuations;
- remove digits;
- replace runs of whitespace, i.e. more than one in a row, with no space; and
- make lowercase.

Apply these changes to the `review_text` column in the `runway` dataset. 

Save as `review_text_cleaned`.

In [30]:
# Remove the forward-slash character


# Remove punctuation


# Remove digits


# Remove running spaces


# Make the text lowercase


## Task 3: Sentiment Analysis on `review_text_cleaned`

Great! The review text is now clean from extraneous characters. You are ready to start performing some NLP tasks!

You will start with sentiment analysis, or the categorizing of text into two buckets - negative or positive. Some models on Hugging Face have the ability to categorize text as neutral as well. 

Sentiment analysis is a common, and useful, NLP task performed in a typical business settings. It can help decision makers determine trends in product or service perception, customer experience, reviews, and more. It can also act as a starting point for deeper analysis.

In this task, you will use the `pipeline()` function with the ["distilbert-base-uncased-finetuned-sst-2-english"](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) model. This is a smaller general-purpose language representation model that is great at text classification (i.e. sentiment analysis).

### Instructions

The `pipeline()` function can help you achieve this.

- Instantiate a new pipeline object for `sentiment-analysis`. Save as `sentimentAnalysis`.
- Use this with a list of the cleaned review text as the input to return sentiment for each review. Save the output as `sent_analysis_output`.

Note that this task is computationally intensive and may take several minutes to run. It runs faster in Premium Workspace.

In [14]:
model = "distilbert-base-uncased-finetuned-sst-2-english"

# Instantiate the new pipeline() object for sentiment analysis. Include the model defined above as the model input.


# Run sentiment analysis on the cleaned review text.


## Task 4: Histogram of Sentiment Score

Well, that was straightfoward, yeah?

As I'm sure you're aware by now, Hugging Face hosts a lot of models which can be employed for a variety of NLP tasks. The sentiment analysis pipeline above used the default model, but you can use any available model to suit your data (as long as they are in the [model hub](https://huggingface.co/models)). For example, [FashionClip](https://huggingface.co/patrickjohncyh/fashion-clip) is a model trained specifically for fashion-related datasets - such as this one - and tasks.

On a related note, the [`pipeline()` module](https://huggingface.co/docs/transformers/main_classes/pipelines#pipelines) is quite versatile as well! It encapsulates multiple transformer pipelines including NLP, computer vision, audio, and multi-modal. Within NLP alone, it covers 12 different tasks such as sentiment analysis (listed as "Text Classification"), Named Entity Recognition, Summarization, Conversations, and Q & A.

We encourage you to further explore this powerful tool.

Now, you're going to understand the distribution of the sentiment of review texts via a histogram. This will give you an overall understanding of the reviews specifically how customers viewed their rented items.

### Instructions

- Parse the output of `sent_analysis_output` into two new columns - `clean_sentiment_category` and `clean_sentiment_score`.
- Use `seaborn` and the new sentiment score column to create a histogram. Set the number of bins to 20.
- Add a title, "Distribution of Sentiment Score", subtitle - "For Clean Review Text" - and x and y labels.

In [17]:
# Parse out the two pieces of output from sent_analysis_output - "label" and "score".


# Create the first histogram using the built-in Pandas histogram method.


## Task 5: Sentiment Over the Years

The distribution is informative from a global perspective - i.e., the reviews skew positive ("great customer experience"). However, it doesn't tell a complete story.

Now that you have sentiment, you can explore the trends of sentiment across different facets of the data (i.e. over time, between products, etc.). Dissecting the data in this way, based on the business questions of interest, will explain more about what is going well and what needs improvement.

### Instructions

In this task, you'll explore how sentiment changed over the years through a bar plot visualization. Feel free to explore other facets as well, if interested.

- Create a new column called `year`.
- Count the number of reviews by `year` and `clean_sentiment_category`. Save as `chart_data`.
- Visualize the count of reviews using a seaborn `.barplot()`.
- Add an appropriate title and axes labels.

In [18]:
# Create a new column called "year" which is the year of the "review_date".


# Create a new dataframe of review counts by "year" and "cleaned_sentiment_category".


# Create a bar plot showing the count of reviews for each sentiment category over the years.


## Task 6: Does Cleaning Text Matter?

Wow - that's quite a set of positive reviews! It seems like the amount of positive reviews increased dramatically in 2021. Potentially a revamp of the customer experience, product quality, more customers, or something else we don't have data for. 

From a business context, digging into the common words/n-grams of the positive reviews can be revealing as to what customers love about the products. Doing the same for the negative reviews can be just as informative for what they don't love (i.e. sizing issues, rental service, etc.). 

We will look into that soon. But first, let's test the previous comment on preprocessing text. Does it matter for sentiment analysis?

In this task, you will build another sentiment analysis pipeline but, in this case, for the non-cleaned `review_text`. Then, you will compare the output from the two pipelines to understand if the cleaning made a difference in categorizing sentiment.

### Instructions

- Duplicate the steps from the previous exercise to run a sentiment analysis pipeline. Save as `sent_analysis_2`.
- Parse the output and save as `sentiment_category` and `sentiment_score`.
- Create a confusion matrix using `pd.crosstab()` comparing the output of `clean_sentiment_category` and `sentiment_category`.

Note that this task is computationally intensive and may take several minutes to run. It runs faster in Premium Workspace.

In [19]:
# Create a new sentiment analysis output for "review_text".


# Parse the output into "sentiment_category" and "sentiment_score".


# Print the confusion matrix comparing the two sentiment category outputs.


Based on the previous results, the sentiment of the review was pretty much the same with cleaned text. Good for us! We can use the original text to perform further analysis. Again, this is situation dependent, so be sure to think through the benefits of cleaning vs. leaving the text as is. 

Transformers, such as those on Hugging Face, are a component of machine learning / deep learning models which are designed to learn sequential data. Using a concept known as "self-attention", they use other tokens (words) in the sequence (sentence) to build an understanding of a specific token (word).

All of this to say (in a simplified manner) transformers, specifically those trained on VERY large datasets, work well on unstructured, un-cleaned text because it takes the whole sentence into context. And every piece adds to that context [see BERT paper here that argues this point](https://arxiv.org/pdf/1904.07531.pdf).

This can be particularly powerful from 1) a time-savings perspective and 2) enabling more unstructured text to be leveraged. With less hassle of data preparation, more data can be used to make decisions. Again, make sure you evaluate the use case and determine if the NLP results are meeting performance expectations.

## Task 7: Embeddings

In the next task, we will switch gears to exploring another common use case for Hugging Face and transformers - text embeddings. 

Embeddings, in a very simplistic definition, are a vector (numerical) representation of something within n-dimensions. In this case, they are text embeddings. (Note: each transformer model will have a different number of dimensions for its results). Embeddings are useful because they represent human language to computers which enables a more sophisticated execution of similarity, text generation, semantic search, and the like. This can be extremely valuable for business tasks such as recommendations and searching within websites or products.

You will now try this out on the `rented for` column using the `sentence_transformer` [package](https://huggingface.co/sentence-transformers). The model you'll use is the BERT-based ["all-MiniLM-L6-v2" model](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) which transformer that maps sentences and paragraphs to an n-dimensional vector.

### Instructions

- Instantiate a new instance of `SentenceTransformer` using the specified model (`model_id`). Save as `model`.
- Generate an embedding for each review text using `model.encode()`. Save as `embeddings`.
- Print the shape of the new embeddings array.

In [46]:
model_id = "sentence-transformers/all-MiniLM-L6-v2"

# Instantiate a new SentenceTransformer object.


# Generate the embeddings for the "rented for" column.




## Task 8: Clustering

The all-MiniLM-L6-v2 model outputs embeddings of 384 dimensions. A higher-dimensional embedding can capture more of the relationship between words which is fantastic!

However, for some tasks, such as clustering, it can be a challenge. Clustering tends to perform poorly with higher dimensions of data (i.e. the Curse of Dimensionality), which poses a common challenge in text mining and NLP. Therefore, dimensionality reduction is often employed in order to calculate distance (e.g. Euclidean) between embeddings, then calculate clusters.

Since the transformer is constructed in such a way to be better at learning context, text - words, sentences, documents - that are similar should have similar vectors and therefore be closer together (i.e. in the same cluster).

Understanding which embeddings are closer together can then be used to determine which products, users, reviews, etc. are similar to each other. This is an important step in common method for building recommendations.

### Instructions

In this task, you'll conduct dimensionality reduction using the `TSNE` module from the `sklearn` package.

- Import `TSNE` from `sklearn.manifold`.
- Create a new TSNE object using the defined input parameters. Save as `tsne`.
- Fit the `tsne` model with the saved embeddings. Save this as `tsne_dims`.

In [47]:
# Import TSNE


# Instantiate a new TSNE object. 


# Generate the tsne dimensions using the saved embeddings.




## Task 9: Visualizing the Clusters

Now each embedding is 2-dimensional. You don't need to be so drastic in dimensionality reduction for clustering, i.e. use explained variance ratio to determine the right number of components, but is important for visualizing the clusters.

With these smaller vectors, let's move on to generating basic clusters (the average of each category) and building a visualization for them!

### Instructions

- Extract the x and y dimensions from the `tsne_dims` result. Save as new columns in `runway` named `x` and `y`, respectively.
- Use `seaborn.scatterplot` to create a new scatterplot using the x and y dimensions and `category` as the "hue".
- Set the title as `Embeddings visualized for 'rented for' reasons and category using t-SNE`.

In [48]:
# Parse out the x and y dimensions from the tsne output.


# Create the scatterplot


## Task 10: Semantic Search

The plot is showing us all of the review embeddings - based on the "rented_for" column - in a two-dimensional space (since our eyes can't handle 384 dimensions)! The color is indicating the type of item rented, i.e. "dress", "gown", "sheath".

There are clearly two main clusters in this dataset for "rented for" - the mainly blue cluster and the mainly orange one. The blue cluster consists primarily of gowns while the other is mainly dresses.

This is a simple example yet still very informative about how customers rent items for which types of occasions.

Embeddings are not only cool to visualize, as mentioned above, they also facilitate "semantic search". Semantic search denotes "searching with meaning" and with context when available. The goal is to infer what the user's intent is then find the most relevant results. This is different from "lexical search" where a search engine looks for literal matches of a query (or defined variants). Both have their strengths and weaknesses, which we encourage you to research further.

Semantic search can be incredibly powerful in a customer experience setting. Specifically, they can help with recommendations (i.e. which product is similar based on reviews, descriptions, etc.) and - obviously - search (i.e. find me products with these phrases or words in their reviews). 

### Instructions

In this task, you'll build a simple semantic search pipeline using the `sentence-transformer` package.

- First, generate an embedding of the pre-defined query using the same model object saved above. Save this query embedding as `query_emb`.
- With the `semantic_search()` function, use the query embedding and the other embeddings to get the top three "hits", i.e. most similar items by review.
- Use a for-loop to print the category and rented for reason for each hit.

In [51]:
# Define query
query = "a gorgeous and flattering dress"

# Embed query


# Calculate similarity between query and item embeddings


# Print top similar items


## Task 11: Generate New Marketing Material

The semantic search returned three different items that appear to be similar to what we were searching for, i.e. "the dress was gorgeous" and "this dress was a great fit". With some fine-tuning, you can achieve even greater performance. We can also notice some constructive feedback on other parts of the dress. Further investigation here would reveal important insights for the business team.

Let's again switch gears for this final task. Let's generate some new text based on a prompt. Specifically, you want to create some brainstorming material for a new marketing campaign. This type of process can save a lot of time in the creation process - helping you start at "I have a good idea" vs. "I need an idea". 

### Instructions

In this task, you'll switch back to the `pipeline()` module.

- Instantiate a text generation pipeline using `model = "gpt2"` as the input parameter. Save as `generator`.
- Using this generator, create 1 new piece of text based on the pre-defined prompt. Save as `output`.
- Print the prompt and the output.

In [29]:
# Instantiate the pipeline for generating text


# Pre-defined prompt
prompt = "New for this winter season, a lovely dress that"

# Use the prompt as input to the generator to return output.


# Print the generated text.
